In [ ]:
%%Shell 
bedtools intersect   -a mm10.5kb_windows.bed   -b gencode.vM17.protein_coding.tr.tss.bed  -wa -wb  | awk -v OFS='\t' '{print substr($1,4)":"$2"-"$3,$7}' |uniq> tmp

In [5]:
require(tidyverse)
require(data.table)
#require(GenomeRange)
require(rtracklayer)
require(IRanges)

In [6]:
gtf <- fread('~/data/GENOME/mm10/gencode.vM17.annotation.gtf',skip = 5) # 1-base 

## make tr tss

In [7]:
gtf.protein_codeing.tr <- gtf %>% filter(V3 == "transcript" & grepl("protein_coding", 
    V9)) %>% mutate(V9 = sub("gene_name \"(.*)\"", "\\1", grep("gene_name", unlist(strsplit(V9, 
    split = ";")), value = T))) %>% select(V1, V4, V5, V9, V6, V7) %>% distinct()

In [15]:
gtf.protein_codeing.tr.tss <- (gtf.protein_codeing.tr) %>% mutate(V4 = V4 - 1, V5 = V5)
for (i in 1:nrow(gtf.protein_codeing.tr.tss)) {
    if (gtf.protein_codeing.tr.tss$V7[i] == "-") {
        gtf.protein_codeing.tr.tss$V4[i] = gtf.protein_codeing.tr.tss$V5[i] - 1
    } else (gtf.protein_codeing.tr.tss$V5[i] = gtf.protein_codeing.tr.tss$V4[i] + 1)
}

In [26]:
head(gtf.protein_codeing.tr.tss)
head(gtf.protein_codeing.tr.tss%>%filter(V7=="+"),n=20)

V1,V4,V5,V9,V6,V7
chr1,3216343,3216344,Xkr4,.,-
chr1,3215631,3215632,Xkr4,.,-
chr1,3671497,3671498,Xkr4,.,-
chr1,4409240,4409241,Rp1,.,-
chr1,4409240,4409241,Rp1,.,-
chr1,4409186,4409187,Rp1,.,-


V1,V4,V5,V9,V6,V7
chr1,4807787,4807788,Lypla1,.,+
chr1,4807822,4807823,Lypla1,.,+
chr1,4807829,4807830,Lypla1,.,+
chr1,4807895,4807896,Lypla1,.,+
chr1,4807897,4807898,Lypla1,.,+
chr1,4807910,4807911,Lypla1,.,+
chr1,4808236,4808237,Lypla1,.,+
chr1,4836904,4836905,Lypla1,.,+
chr1,4807891,4807892,RP24-426M1.3,.,+
chr1,4857813,4857814,Tcea1,.,+


In [23]:
write.table(gtf.protein_codeing.tr.tss, "~/data/GENOME/mm10/gencode.vM17.protein_coding.tr.tss.bed", sep = '\t',
    row.names = F, col.names = F, quote = F)

## Gene

In [25]:
dim(gtf.protein_codeing)

[1] 21981     6

In [24]:
gtf.protein_codeing <- gtf %>% filter(V3 == "gene" & grepl("protein_coding", V9)) %>% mutate(V9 = sub("gene_name \"(.*)\"", 
    "\\1", grep("gene_name", unlist(strsplit(V9, split = ";")), value = T))) %>% 
    select(V1, V4, V5, V9, V6, V7)
head(gtf.protein_codeing)


V1,V4,V5,V9,V6,V7
chr1,3205901,3671498,Xkr4,.,-
chr1,3999557,4409241,Rp1,.,-
chr1,4490931,4497354,Sox17,.,-
chr1,4773206,4785739,Mrpl15,.,-
chr1,4807788,4848410,Lypla1,.,+
chr1,4807892,4886770,RP24-426M1.3,.,+


In [21]:
write.table(gtf.protein_codeing, "~/data/GENOME/mm10/gencode.vM17.protein_coding.gene.bed", sep = '\t',
    row.names = F, col.names = F, quote = F)

## make 5kb window overlapped with tss

In [9]:
gtf.protein_codeing <- import("~/data/GENOME/mm10/gencode.vM17.protein_coding.tr.tss.bed")
windows.5k <- import('~/data/GENOME/mm10/mm10.5kb_windows.bed')

In [11]:
head(windows.5k,1)
head(gtf.protein_codeing)

GRanges object with 1 range and 0 metadata columns:
      seqnames    ranges strand
         <Rle> <IRanges>  <Rle>
  [1]     chr1    1-5000      *
  -------
  seqinfo: 22 sequences from an unspecified genome; no seqlengths

GRanges object with 6 ranges and 2 metadata columns:
      seqnames    ranges strand |        name     score
         <Rle> <IRanges>  <Rle> | <character> <numeric>
  [1]     chr1   3216343      - |        Xkr4      <NA>
  [2]     chr1   3215631      - |        Xkr4      <NA>
  [3]     chr1   3671497      - |        Xkr4      <NA>
  [4]     chr1   4409240      - |         Rp1      <NA>
  [5]     chr1   4409240      - |         Rp1      <NA>
  [6]     chr1   4409186      - |         Rp1      <NA>
  -------
  seqinfo: 22 sequences from an unspecified genome; no seqlengths

In [5]:
hits <- findOverlaps(windows.5k,gtf.protein_codeing)
hits

Hits object with 97436 hits and 0 metadata columns:
          queryHits subjectHits
          <integer>   <integer>
      [1]       644           2
      [2]       644           1
      [3]       735           3
      [4]       873           7
      [5]       882           6
      ...       ...         ...
  [97432]    544823       97500
  [97433]    544824       97501
  [97434]    544824       97502
  [97435]    544824       97503
  [97436]    544834       97504
  -------
  queryLength: 545015 / subjectLength: 97517

In [13]:
windows.5k.prom<- windows.5k[queryHits(hits)]
windows.5k.prom$name <- gtf.protein_codeing$name[subjectHits(hits)]
length(windows.5k.prom)
windows.5k.prom<-(windows.5k.prom%>%as.data.frame()%>%distinct())
dim(windows.5k.prom)

[1] 97436

[1] 43790     6

In [17]:
head(windows.5k.prom)

seqnames,start,end,width,strand,name
chr1,3215001,3220000,5000,*,Xkr4
chr1,3670001,3675000,5000,*,Xkr4
chr1,4360001,4365000,5000,*,Rp1
chr1,4405001,4410000,5000,*,Rp1
chr1,4490001,4495000,5000,*,Sox17
chr1,4495001,4500000,5000,*,Sox17


In [30]:
windows.5k.prom <- mutate_if(windows.5k.prom, is.numeric, as.integer)
windows.5k.prom %>% mutate(start = start - 1) %>% filter(name == " Jph1") %>% unite("V1", 
    1:2, sep = ":") %>% unite("V1", 1:2, sep = "-")

V1,width,strand,name
chr1:1.7e+07-17005000,5000,*,Jph1
chr1:17095000-17100000,5000,*,Jph1


In [8]:
)
windows.5k.prom<-(windows.5k.prom %>% mutate(seqnames = sub("chr", "", seqnames), start = start - 
    1)%>%unite("V1",1:2,sep = ":"))%>%unite("V1",1:2,sep="-")

In [10]:
op.regions <- fread('./JYH_854_1_2/JYH_854_1_2.regions',header = F)

In [11]:
head(op.regions)

V1
1:10000000-10005000
1:100000000-100005000
1:100005000-100010000
1:100010000-100015000
1:100015000-100020000
1:100020000-100025000


In [12]:
sum(op.regions$V1 %in% windows.5k.prom$V1)
nrow(windows.5k.prom)
head(windows.5k.prom)


write.table(windows.5k.prom[, c(1, 4)], "~/data/GENOME/mm10/gencode.vM17.5kb_tr_promoter_names.txt", 
    sep = "\t", row.names = F, col.names = F, quote = F)

[1] 40999

[1] 43790

V1,width,strand,name
1:3215000-3220000,5000,*,Xkr4
1:3670000-3675000,5000,*,Xkr4
1:4360000-4365000,5000,*,Rp1
1:4405000-4410000,5000,*,Rp1
1:4490000-4495000,5000,*,Sox17
1:4495000-4500000,5000,*,Sox17
